In [1]:
import pandas as pd
import pygsheets
import numpy as np

In [21]:
gc = pygsheets.authorize(outh_file='/Users/irarickman/client_secret.json')
file = 'Activity data'
sh = gc.open(file)

wks = sh[0]
strav=wks.get_as_df(empty_value=np.nan)

In [43]:
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as BS
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.chrome.options import Options 
import configparser
import time
import datetime
import swagger_client
from swagger_client.rest import ApiException
import pygsheets
import math


#authorization
gc = pygsheets.authorize(outh_file='/Users/irarickman/client_secret.json')
file = 'Activity data'
sh = gc.open(file)

wks_1 = sh[0]
strava=wks_1.get_as_df(empty_value=np.nan)
strava.dropna(how='all',inplace=True)

wks_2=sh[1]
whoop=wks_2.get_as_df(empty_value=np.nan)
whoop.dropna(how='all',inplace=True)
whoop.dropna(axis=1,how='all',inplace=True)


In [36]:
## reformatting the datasets
mult_mile=0.000621371
strava['miles']=strava.distance*mult_mile
strava['race']=strava.workout_type.apply(lambda x: 1 if x in [1.0,11.0] else 0 )
strava['date_string']=strava.start_date.apply(lambda x: x[:10])
strava['moving_minutes']=strava.moving_time/60
strava['elapsed_minutes']=strava.elapsed_time/60
strava['rest']=strava.elapsed_minutes-strava.moving_minutes
## average speed is in meters/second - 2.237 to multiply to mph
strava['avg_mph']=strava.average_speed*2.237
strava['time_since_last_act']=(pd.to_datetime(strava.start_date)-pd.to_datetime(strava.start_date.shift(-1))).astype('timedelta64[h]')
strava.start_date=pd.to_datetime(strava.start_date)
strava.sort_values('start_date',inplace=True)
strava['order']=strava.groupby('date_string').start_date.rank()

In [45]:
whoop['recovery']=whoop['recovery'].astype(str).apply(lambda x: np.nan if '%' not in x else float(x[:len(x)-1])/100)
whoop['sleep_perf']=whoop['sleep_perf'].astype(str).apply(lambda x: np.nan if '-' in x or "na" in x else float(x[:len(x)-1])/100)
whoop['rec_color']=whoop.recovery.apply(lambda x: 'red' if x<.34 else ('yellow' if x<.67 else ('none' if np.isnan(x) else 'green') ))
whoop.strain=whoop.strain.apply(lambda x: float(x) if x!="---" else 0)
def time_to_dec(t):
    if ":" in t:
        hr=float(t[:t.find(':')])
        m=float(t[t.find(':')+1:])/60
        return hr+m
    else:
        return np.nan
whoop.sleep=whoop.sleep.astype(str).apply(time_to_dec)
whoop.rec_sleep=whoop.rec_sleep.astype(str).apply(time_to_dec)
whoop['sleep_addition']=whoop.rec_sleep-7.75
whoop['pday_rec']=whoop['recovery'].shift(-1)
whoop['pday_rec_col']=whoop['rec_color'].shift(-1)
for n in range(1,4):
    act_num="activity_{}_score".format(n)
    whoop[act_num]=whoop[act_num].apply(lambda x: np.nan if x=='null' else x)
whoop['activity_total']=whoop[['activity_1_score','activity_2_score','activity_3_score']].apply(lambda x: sum([0 if np.isnan(y) else 1 for y in x ]),axis=1 )
whoop['pday_acts']=whoop['activity_total'].shift(-1)
whoop['pday_strain']=whoop['strain'].shift(-1)
whoop['pday_sleep']=whoop.sleep.shift(-1)
whoop['pday_sleep_perf']=whoop.sleep_perf.shift(-1)
whoop['rolling_prev_2']=whoop.pday_sleep.rolling(2).mean()
whoop['prev_strain_rec_gap']=whoop.pday_strain/whoop.pday_strain.max()-whoop.pday_rec

In [ ]:
whoop['recovery']=whoop['recovery'].astype(str).apply(lambda x: np.nan if '%' not in x else float(x[:len(x)-1])/100)
whoop['sleep_perf']=whoop['sleep_perf'].astype(str).apply(lambda x: np.nan if '-' in x or "na" in x else float(x[:len(x)-1])/100)
whoop['rec_color']=whoop.recovery.apply(lambda x: 'red' if x<.34 else ('yellow' if x<.67 else ('none' if np.isnan(x) else 'green') ))
whoop.strain=whoop.strain.apply(lambda x: float(x) if x!="---" else 0)
def time_to_dec(t):
    if ":" in t:
        hr=float(t[:t.find(':')])
        m=float(t[t.find(':')+1:])/60
        return hr+m
    else:
        return np.nan
whoop.sleep=whoop.sleep.astype(str).apply(time_to_dec)
whoop.rec_sleep=whoop.rec_sleep.astype(str).apply(time_to_dec)
whoop['sleep_addition']=whoop.rec_sleep-7.75
whoop['pday_rec']=whoop['recovery'].shift(-1)
whoop['pday_rec_col']=whoop['rec_color'].shift(-1)
for n in range(1,4):
    act_num="activity_{}_score".format(n)
    whoop[act_num]=whoop[act_num].apply(lambda x: np.nan if x=='null' else x)
whoop['activity_total']=whoop[['activity_1_score','activity_2_score','activity_3_score']].apply(lambda x: sum([0 if np.isnan(y) else 1 for y in x ]),axis=1 )
whoop['pday_acts']=whoop['activity_total'].shift(-1)
whoop['pday_strain']=whoop['strain'].shift(-1)
whoop['pday_sleep']=whoop.sleep.shift(-1)
whoop['pday_sleep_perf']=whoop.sleep_perf.shift(-1)
whoop['rolling_prev_2']=whoop.pday_sleep.rolling(2).mean()
whoop['prev_strain_rec_gap']=whoop.pday_strain/whoop.pday_strain.max()-whoop.pday_rec

In [47]:
act_1=all_whoop[['date','activity_1','activity_1_score']]
act_1.columns=['date','activity','score']
act_1['order']=1

act_2=all_whoop[['date','activity_2','activity_2_score']]
act_2.columns=['date','activity','score']
act_2['order']=2
act_2.dropna(inplace=True)

act_3=all_whoop[['date','activity_3','activity_3_score']]
act_3.columns=['date','activity','score']
act_3['order']=3
act_3.dropna(inplace=True)

full_whoop=pd.concat([act_1,act_2,act_3])
full_desc=whoop.drop(['activity_1','activity_2','activity_3','activity_1_score','activity_2_score','activity_3_score'],
                   axis=1)
whoop_df=full_desc.merge(full_whoop, how ='left',left_on='date',right_on='date')
whoop_df.head()

/Users/irarickman/anaconda2/envs/py35/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/irarickman/anaconda2/envs/py35/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/irarickman/anaconda2/envs/py35/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

,strain,recovery,sleep_perf,sleep,rec_sleep,date,date_string,rec_color,sleep_addition,pday_rec,...,activity_total,pday_acts,pday_strain,pday_sleep,pday_sleep_perf,rolling_prev_2,prev_strain_rec_gap,activity,score,order
0,0.0,NaN,NaN,NaN,NaN,"Today, May 4th",5/4/2019,none,NaN,0.66,...,0,3.0,17.0,5.500000,0.67,NaN,0.161256,null,NaN,1
1,17.0,0.66,0.67,5.500000,8.266667,"Fri, May 3rd",5/3/2019,yellow,0.516667,0.57,...,3,0.0,10.0,8.833333,0.93,7.166667,-0.086908,Running,8.2,1
2,17.0,0.66,0.67,5.500000,8.266667,"Fri, May 3rd",5/3/2019,yellow,0.516667,0.57,...,3,0.0,10.0,8.833333,0.93,7.166667,-0.086908,Functional Fitness,9.2,2
3,17.0,0.66,0.67,5.500000,8.266667,"Fri, May 3rd",5/3/2019,yellow,0.516667,0.57,...,3,0.0,10.0,8.833333,0.93,7.166667,-0.086908,Running,10.4,3
4,10.0,0.57,0.93,8.833333,9.500000,"Thu, May 2nd",5/2/2019,yellow,1.750000,0.29,...,0,1.0,14.2,6.816667,0.75,7.825000,0.395990,null,NaN,1


In [49]:
wks_1.set_dataframe(strava,(1,1))
wks_2.set_dataframe(whoop,(1,1))
wks_3=sh[2]
wks_3.set_dataframe(whoop_df,(1,1))

In [5]:
test_whoo=whoo
test_whoo.drop_duplicates('date_string',inplace=True)
test_whoo

,strain,recovery,sleep_perf,sleep,rec_sleep,date,date_string,activity_1,activity_1_score,activity_2,activity_2_score,activity_3,activity_3_score
0,---,---,---%,---,---,"Today, Apr 16th",4/16/2019,null,null,null,null,null,null
1,9.3,25%,94%,6:45,7:09,"Mon, Apr 15th",4/15/2019,null,null,null,null,null,null
2,10.2,20%,97%,8:07,8:22,"Sun, Apr 14th",4/14/2019,null,null,null,null,null,null
5,15.3,59%,100%,8:03,7:10,"Sat, Apr 13th",4/13/2019,Running,13.3,null,null,null,null
6,18.8,65%,40%,3:49,9:34,"Fri, Apr 12th",4/12/2019,Running,18,null,null,null,null
7,16.5,44%,79%,7:01,8:51,"Thu, Apr 11th",4/11/2019,Running,14.3,null,null,null,null
8,4.9,---,---%,---,---,"Wed, Apr 10th",4/10/2019,null,null,null,null,null,null
9,15.2,42%,72%,6:26,8:57,"Tue, Apr 9th",4/9/2019,Running,13.3,null,null,null,null
10,11.4,18%,81%,7:31,9:16,"Mon, Apr 8th",4/8/2019,null,null,null,null,null,null
11,20.5,54%,94%,7:30,8:00,"Sun, Apr 7th",4/7/2019,Running,19.3,null,null,null,null


In [ ]:
all_whoop['recovery']=all_whoop['recovery'].astype(str).apply(lambda x: np.nan if '%' not in x else float(x[:len(x)-1])/100)
all_whoop['sleep_perf']=all_whoop['sleep_perf'].astype(str).apply(lambda x: np.nan if '-' in x or "na" in x else float(x[:len(x)-1])/100)
all_whoop['rec_color']=all_whoop.recovery.apply(lambda x: 'red' if x<.34 else ('yellow' if x<.67 else ('none' if np.isnan(x) else 'green') ))
all_whoop.strain=all_whoop.strain.apply(lambda x: float(x) if x!="---" else 0)
def time_to_dec(t):
    if ":" in t:
        hr=float(t[:t.find(':')])
        m=float(t[t.find(':')+1:])/60
        return hr+m
    else:
        return np.nan
all_whoop.sleep=all_whoop.sleep.astype(str).apply(time_to_dec)
all_whoop.rec_sleep=all_whoop.rec_sleep.astype(str).apply(time_to_dec)
all_whoop['sleep_addition']=all_whoop.rec_sleep-7.75
all_whoop['pday_rec']=all_whoop['recovery'].shift(-1)
all_whoop['pday_rec_col']=all_whoop['rec_color'].shift(-1)
for n in range(1,4):
    act_num="activity_{}_score".format(n)
    all_whoop[act_num]=all_whoop[act_num].apply(lambda x: np.nan if x=='null' else x)
all_whoop['activity_total']=all_whoop[['activity_1_score','activity_2_score','activity_3_score']].apply(lambda x: sum([0 if np.isnan(y) else 1 for y in x ]),axis=1 )
all_whoop['pday_acts']=all_whoop['activity_total'].shift(-1)
all_whoop['pday_strain']=all_whoop['strain'].shift(-1)
all_whoop['pday_sleep']=all_whoop.sleep.shift(-1)
all_whoop['pday_sleep_perf']=all_whoop.sleep_perf.shift(-1)
all_whoop['rolling_prev_2']=all_whoop.pday_sleep.rolling(2).mean()
all_whoop['prev_strain_rec_gap']=all_whoop.pday_strain/all_whoop.pday_strain.max()-all_whoop.pday_rec


In [8]:
wks_2.set_dataframe(test_whoo,(1,1))

In [144]:
from selenium.webdriver.chrome.options import Options as cOptions

In [145]:
testopts=cOptions()

In [140]:
def get_strava(last_date=False):
    ## Getting log in info
    config = configparser.ConfigParser()
    config.read("../strava.ini")
    params={'client_id':config['strava']['client_id'],
            'client_secret':config['strava']['client_secret'],
            'code':config['strava']['code']}
    auth_url=config['strava']['auth_url']
    ref_url=config['strava']['ref_url']
    athlete_id=config['strava']['athlete_id']
    
    if not last_date:
        ## Getting the most recent date
        last_date=datetime.datetime.strptime(strav.start_date.max().split(' ')[0],"%Y-%m-%d")
    else:
        last_date=datetime.datetime.strptime(last_date,"%Y-%m-%d")
    timestamp=last_date.timestamp()
    delta=datetime.datetime.now() - last_date
    date_diff=delta.days+5

    r_auth = requests.post(auth_url, data=params)
    response=r_auth.json()

    configuration = swagger_client.Configuration()
    configuration.access_token = response['access_token']

    # create an instance of the API class
    api_instance = swagger_client.ActivitiesApi(swagger_client.ApiClient(configuration))
    if date_diff<200:
        try:
            # Get Authenticated Athlete
            api_response = api_instance.get_logged_in_athlete_activities(after=timestamp, per_page=date_diff)
        except ApiException as e:
            print("Exception when calling AthletesApi->get_logged_in_athlete: {}\n".format(e))
    else:
        num_rounds=math.ceil(date_diff/200)
        api_response=[]
        for n in range(num_rounds):
            if n==num_rounds:
                page_num=date_diff-(200*(n-1))
            else:
                page_num=200
            try:
                # Get Authenticated Athlete
                activities = api_instance.get_logged_in_athlete_activities(after=timestamp, page=n+1,per_page=page_num)
            except ApiException as e:
                print("Exception when calling AthletesApi->get_logged_in_athlete: {}\n".format(e))
            api_response=api_response+activities
                
    example=list(api_response[len(api_response)-1].to_dict().keys())
    example=[x for x in example if x not in ['map','athlete','start_latlng','end_latlng']]
    dicts={}
    for n in range(len(api_response)):
        d=api_response[n].to_dict()
        new_dict={variable:d[variable] for variable in example}
        dicts[n]=new_dict

    index=list(dicts.keys())
    new_acts=pd.DataFrame([dicts[key] for key in index],index=index)
    if len(strav)==0:
        wks_1.set_dataframe(new_acts,(1,1))
    else:
        all_acts=pd.concat([new_acts,strav])
        all_acts.drop_duplicates(inplace=True)
        wks_1.set_dataframe(all_acts,(1,1))

In [161]:
def get_whoop(most_recent=False):
    
    today=datetime.datetime.today()
    config = configparser.ConfigParser()
    config.read("../strava.ini")
    
    w='https://app.whoop.com/login'
    
    chrome_options = Options()  
    chrome_options.add_argument("--headless")  
    chrome_options.binary_location = '/Applications/Google Chrome Canary.app/Contents/MacOS/Google Chrome Canary'

    browser=webdriver.Chrome('/usr/local/bin/chromedriver',chrome_options=chrome_options)
    browser.get(w)
    
    username=browser.find_element_by_name('username')
    password=browser.find_element_by_name('password')
    username.send_keys(config['whoop']['username'])
    password.send_keys(config['whoop']['pass'])
    submit=browser.find_element_by_name('form')
    submit.click()
    
    def go_to_strain():
        work=0
        while work==0:
            try:
                strain_click=browser.find_elements_by_class_name('score')[0]
                strain_click.click()
                work=1
            except (StaleElementReferenceException,IndexError):
                work=0
        
    ## goal function is tos look for the activity - if it doesn't exist, then look for "no activity" 
## store the result, if the old activity strain is the same as the new

    def get_activity_strain(old_strain,old_date):
        work=0
        while work==0:
            try:
                buttons=browser.find_elements_by_tag_name('button')
                new_strain=[x.text for item in [x for x in [span.find_elements_by_tag_name('span') for span in [b for b in buttons if b.get_attribute("ng-click")=="click($event, activity.id)"]]] for x in item]
                if len(new_strain)==0:
                    new_strains=["null"]*6
                elif len(new_strain)<7:
                    new_strains=new_strain+['null']*(6-len(new_strain))
                else:
                    new_strains=new_strain
                if old_strain!=new_strains:
                    return new_strains
                    work=1
                elif old_date!=get_date(old_date):
                    return new_strains
                    work=1
            except StaleElementReferencException:
                work=0
                 
    def get_date(old_date):
        work=0
        while work==0:
            try:
                new_day=browser.find_element_by_class_name('datepicker--label')
                new_date=new_day.text
                if new_date!=old_date:
                    return [new_date]
                    work=1
            except StaleElementReferenceException:
                work=0
            
    def get_scores(old_scores):
        work=0
        while work==0:
            try:
                new_scores=browser.find_elements_by_class_name('score')
                new_score_list=[x.text for x in new_scores]
                if new_score_list!=old_scores:
                    return new_score_list
                    work=1
            except StaleElementReferenceException:
                work=0
        
    def next_page(tod):
        yesterday=tod-datetime.timedelta(1)
        yesterdate=yesterday.strftime('%Y-%m-%d')
        browser.get('https://app.whoop.com/athlete/24590/strain/1d/{}'.format(yesterdate))
        tod=yesterday
#     def get_button():
#         work=0
#         while work==0:
#             try: 
#                 #back=browser.find_element_by_class_name('datepicker--prev')
#                 back=browser.find_element_by_xpath('/html/body/ui-view/div[4]/main/ui-view/div/ui-view/div/div[2]/div/datepicker/div/button[1]')
#                 back.click()
#                 #back.send_keys(Keys.ENTER)
#                 work=1
#             except StaleElementReferenceException:
#                 work=0

    if not most_recent:
        most_recent=whoo.date[0].split(', ')[1]
    
    new_whoop=pd.DataFrame(columns=['strain','recovery','sleep_perf','sleep','rec_sleep','date', 
                            'activity_1','activity_1_score','activity_2','activity_2_score',
                           'activity_3','activity_3_score'])
    num=0
    old_date='today'
    old_scores=['none']
    old_strain=['null']
    go_to_strain()
    new_date=get_date(old_date)
    while (new_date[0].split(', ')[1].strip().lower()!=most_recent and num<20):
        new_date=get_date(old_date)
        new_score=get_scores(old_scores)
        strains=get_activity_strain(old_strain,old_date)
        old_date=new_date
        old_scores=new_score
        old_strain=strains
        new_whoop.loc[len(new_whoop)]=new_score[:5]+new_date+strains
        next_page(today)
        num+=1
    print(new_whoop.columns, whoo.columns)
#     all_whoop=pd.concat([new_whoop,whoo])
#     all_whoop.drop_duplicates(inplace=True)
#     if len(all_whoop)>=len(whoo):
#         wks_2.set_dataframe(all_whoop,(1,1))

In [158]:
tod=datetime.datetime.today()
yesterday=tod-datetime.timedelta(1)
yesterdate=yesterday.strftime('%Y-%m-%d')
print(yesterdate,tod)

2019-03-31 2019-04-01 22:34:42.366848


In [125]:
get_strava(last_date='2018-04-01')

In [162]:
get_whoop()

/Users/irarickman/anaconda2/envs/py35/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"class name","selector":"datepicker--label"}
  (Session info: headless chrome=75.0.3753.2)
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.14.3 x86_64)


In [90]:
strav.isnull().sum()

achievement_count         0
athlete_count             0
average_speed             0
average_watts             6
comment_count             0
commute                   0
device_watts              6
distance                  0
elapsed_time              0
elev_high                 0
elev_low                  0
external_id               0
flagged                   0
gear_id                   6
has_kudoed                0
id                        0
kilojoules                6
kudos_count               0
manual                    0
max_speed                 0
max_watts                 6
moving_time               0
name                      0
photo_count               0
private                   0
start_date                0
start_date_local          0
timezone                  0
total_elevation_gain      0
total_photo_count         0
trainer                   0
type                      0
upload_id                 0
weighted_average_watts    6
workout_type              6
dtype: int64

In [109]:
import math
math.ceil(450/200)

3

In [113]:
len(strav)

0